In [1]:
#!/usr/bin/env python

"""MXGSTGF_BackgroundObservation_test_Data_read_into_db.py: read raw data in database"""


__author__ 		= "Gareth Murphy"
__copyright__ 	= "Copyright 2016, ASDC, DTU Space"
__credits__ 	= ["spacepy, bitstring, textwrap, numpy"]
__license__ 	= "XXX"
__version__ 	= "0.0.1"
__maintainer__ 	= "Gareth Murphy"
__email__ 		= "gmurphy@space.dtu.dk"
__status__ 		= "Working version, finds and reads data correctly "

import os, sys


home="/Users/garmu/Dropbox/"
proj_path = home+"/asdc/web/"

proj_path = home+"/myenv/mysite/"
# This is so Django knows where to find stuff.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mysite.settings")
sys.path.append(proj_path)

# This is so my local_settings.py gets loaded.
os.chdir(proj_path)

# This is so models get loaded.
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


from asim.models import * 
    

In [2]:
mylist=['06' ,'08' ,'0a' ,'0d' ,'0f' ,'12' ,'16' ,'19' ,'1c' ,'1f' ,'21' ,'25' ,'28' ,'2b' ,'2d' ,'30' ,'34']
mylist=['06']
mylist=['06' ,'08' ,'0a' ,'0d' ,'0f' ,'16' ,'19' ,'1c' ,'1f' ,'21' ,'25' ,'28' ,'2b' ,'2d' ,'30']

for tag in mylist:

    fname='PFM_20160513_0912_raw.txt'

    path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/raw/'


    #fname='PFM_20160513_0912_raw.txt'
    i=0

    import re


    MXGS = '80'
    GROUND = '92'
    SCIENCE_DATA = '02'
    INITIAL_TGF_TM ='a0'
    CONTINU_TGF_TM ='a1'
    INITIAL_BACKGR_TM ='a2'
    CONTINU_BACKGR_TM ='a3'
    INITIAL_PULSE_TM ='a4'
    CONTINU_PULSE_TM ='a5'
    INITIAL_SAMP_TM ='a6'
    CONTINU_SAMP_TM ='a7'
    INITIAL_AUR_TM ='a8'
    CONTINU_AUR_TM ='a9'

    regex = re.compile(MXGS+GROUND+SCIENCE_DATA+'a[23]....00'+tag)



    datalist=[]

    masterdata=[]
    total=0
    import textwrap

    with open (path+fname) as f:    
        for line in f:
            if re.match(regex,line):
                datalist.append(line)
                a=datalist[i]
                dl=textwrap.wrap(a, 4)

                from bitstring import Bits

                f = open('file'+str(i).zfill(3)+'.ccsds', 'wb')
                Bits('0x'+line).tofile(f)
                f.close()


                total =  total + hex2int (a[8:12])
                if ( i > 0) :
                    #print (dl[0:7])
                # delete first 14 bytes/28 hex (Continuation Header)
                # delete first 16 bytes/ 32 hex/ 8 groups of 4 hex characters
                    dl=dl[8:]
                i=i+1
                masterdata.extend(dl)

    
                
    x=MXGSBackgroundObservation()
    background_1second=x.hexstring_to_db(masterdata)
    print x.observation_id
    x.save()
    qq=background_1second
    for i in range(1,60):
    
        y=MXGSBackgroundObservation1Second()
        qq=y.hexstring_to_db(x,qq)
        y.save()
        #pp.pprint(stuff)


0006
0008
000a
000d
000f
0016
0019
001c
001f
0021
0025
0028
002b
002d
0030


In [3]:
import pprint
stuff = y.__dict__
pp = pprint.PrettyPrinter(indent=4)